**Lo primero es cargar el archivo json para utilizar su ruta para poder leer el archivo y obtener los datos.** **FUNDAMENTAL**

In [11]:
file_path = "farmers-protest-tweets-2021-2-4.json"

**Las top 10 fechas donde hay más tweets.**

Desarrollo de Q1 optimizando el tiempo de ejecucion





In [18]:
import json
from collections import defaultdict
from datetime import datetime, date
from typing import List, Tuple

def q1_time(file_path: str) -> List[Tuple[date, str]]:
    date_tweet_counts = defaultdict(lambda: defaultdict(int))

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                tweet = json.loads(line)
                if 'user' in tweet and 'date' in tweet:
                    tweet_date = datetime.strptime(tweet['date'], "%Y-%m-%dT%H:%M:%S%z").date()
                    username = tweet['user']['username']
                    date_tweet_counts[tweet_date][username] += 1
            except (json.JSONDecodeError, KeyError):
                continue

    top_10_dates = sorted(date_tweet_counts.keys(), key=lambda x: sum(date_tweet_counts[x].values()), reverse=True)[:10]

    result = [(tweet_date, max(date_tweet_counts[tweet_date].items(), key=lambda x: x[1])[0]) for tweet_date in top_10_dates]

    return result

file_path = "farmers-protest-tweets-2021-2-4.json"
try:
    top_10_dates_users = q1_time(file_path)
    print(top_10_dates_users)
except FileNotFoundError:
    print(f"File not found: {file_path}. Please check the file path and try again.")
except json.JSONDecodeError as e:
    print(f"Error parsing JSON file: {e}")


[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


Desarrollo de Q1 optimizando el uso de memoria



In [ ]:
!pip install memory_profiler

In [ ]:
import json
from collections import defaultdict
from datetime import datetime, date
from typing import List, Tuple
from memory_profiler import profile

@profile
def q1_memory(file_path: str) -> List[Tuple[date, str]]:
    date_tweet_counts = defaultdict(lambda: defaultdict(int))

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                tweet = json.loads(line)
                tweet_date = datetime.strptime(tweet['date'], "%Y-%m-%dT%H:%M:%S%z").date()
                username = tweet['user']['username']
                date_tweet_counts[tweet_date][username] += 1
            except (json.JSONDecodeError, KeyError, ValueError) as e:
                continue

    top_10_dates = sorted(date_tweet_counts.items(), key=lambda x: sum(x[1].values()), reverse=True)[:10]
    result = [(tweet_date, max(users.items(), key=lambda x: x[1])[0]) for tweet_date, users in top_10_dates]

    return result

file_path = "farmers-protest-tweets-2021-2-4.json"
try:
    top_10_dates_users = q1_memory(file_path)
    print(top_10_dates_users)
except FileNotFoundError:
    print(f"File not found: {file_path}. Please check the file path and try again.")
except json.JSONDecodeError as e:
    print(f"Error parsing JSON file: {e}")



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/memory_profiler.py", line 847, in enable
    sys.settrace(self.trace_memory_usage)



ERROR: Could not find file <ipython-input-6-5714df6985c6>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/memory_profiler.py", line 850, in disable
    sys.settrace(self._original_trace_function)



[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


**Los top 10 emojis más usados con su respectivo conteo.**

Desarrollo de Q2 optimizando el tiempo de ejecucion


In [ ]:
!pip install emoji==1.7

In [17]:
import json
import re
from collections import defaultdict
from typing import List, Tuple

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    emoji_counts = defaultdict(int)
    emoji_pattern = re.compile(r'[\U0001F300-\U0001F64F]|[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F780-\U0001F7FF]|[\U0001F800-\U0001F8FF]', flags=re.UNICODE)

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                tweet = json.loads(line)
                if 'content' in tweet:
                    emojis = emoji_pattern.findall(tweet['content'])
                    for emoji in emojis:
                        emoji_counts[emoji] += 1
            except (json.JSONDecodeError, KeyError):
                continue

    top_10_emojis = sorted(emoji_counts.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_10_emojis

if __name__ == "__main__":
    file_path = "farmers-protest-tweets-2021-2-4.json"
    try:
        top_10_emojis = q2_time(file_path)
        print(top_10_emojis)
    except FileNotFoundError:
        print(f"File not found: {file_path}. Please check the file path and try again.")
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON file: {e}")


[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('🌾', 2363), ('🏻', 2080), ('🏽', 1218), ('👇', 1108), ('💚', 1040), ('💪', 947), ('🏼', 857)]


Desarrollo de Q2 optimizando el uso de memoria

In [ ]:
from collections import defaultdict
from typing import List, Tuple
from emoji import UNICODE_EMOJI
from memory_profiler import profile

def is_emoji(char: str) -> bool:
    # Validar si el carácter es un emoji Unicode (Optimiza el uso de memoria al interpretar emojis directamente con la libreria)
    return char in UNICODE_EMOJI['en']

@profile
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    emoji_counts = defaultdict(int)
    emoji_pattern = re.compile(r'[\U0001F000-\U0001FFFF]|[\U00002000-\U00003FFF]', flags=re.UNICODE)  # Patrón para emojis Unicode

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                tweet = json.loads(line)
                if 'content' in tweet:
                    emojis = emoji_pattern.findall(tweet['content'])
                    valid_emojis = filter(is_emoji, emojis)  # Filtrar solo emojis válidos
                    for emoji in valid_emojis:
                        emoji_counts[emoji] += 1
            except (json.JSONDecodeError, KeyError):
                continue

    top_10_emojis = sorted(emoji_counts.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_10_emojis

if __name__ == "__main__":
    file_path = "farmers-protest-tweets-2021-2-4.json"
    try:
        top_10_emojis = q2_memory(file_path)
        print(top_10_emojis)
    except FileNotFoundError:
        print(f"File not found: {file_path}. Please check the file path and try again.")
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON file: {e}")



ERROR: Could not find file <ipython-input-8-5134f60780bc>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.
[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]


**El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos.**




Desarrollo de Q3 optimizando el tiempo de ejecucion


In [14]:
import json
from collections import defaultdict
import re
from typing import List, Tuple

def extract_mentions(text: str) -> List[str]:
    # Utilizamos regex para encontrar menciones (@username)
    mention_pattern = re.compile(r'@(\w+)')
    return mention_pattern.findall(text)

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    mention_counts = defaultdict(int)

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                tweet = json.loads(line)
                if 'content' in tweet:
                    mentions = extract_mentions(tweet['content'])
                    for mention in mentions:
                        mention_counts[mention] += 1
            except (json.JSONDecodeError, KeyError):
                continue

    top_10_mentions = sorted(mention_counts.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_10_mentions

file_path = "farmers-protest-tweets-2021-2-4.json"
try:
    top_10_mentions = q3_time(file_path)
    print(top_10_mentions)
except FileNotFoundError:
    print(f"File not found: {file_path}. Please check the file path and try again.")
except json.JSONDecodeError as e:
    print(f"Error parsing JSON file: {e}")


[('narendramodi', 2261), ('Kisanektamorcha', 1836), ('RakeshTikaitBKU', 1639), ('PMOIndia', 1422), ('RahulGandhi', 1125), ('GretaThunberg', 1046), ('RaviSinghKA', 1015), ('rihanna', 972), ('UNHumanRights', 962), ('meenaharris', 925)]


Desarrollo de Q3 optimizando el uso de memoria

In [ ]:
import json
import re
from collections import defaultdict
from typing import List, Tuple

def extract_mentions(text: str) -> List[str]:
    # Utilizamos regex para encontrar menciones (@username)
    mention_pattern = re.compile(r'@(\w+)')
    return mention_pattern.findall(text)

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    mention_counts = defaultdict(int)

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                tweet = json.loads(line)
                if 'content' in tweet:
                    # Utilizamos generadores para evitar almacenar todos los tweets en memoria
                    mentions = extract_mentions(tweet['content'])
                    for mention in mentions:
                        mention_counts[mention] += 1
            except (json.JSONDecodeError, KeyError):
                continue

    top_10_mentions = sorted(mention_counts.items(), key=lambda x: x[1], reverse=True)[:10] #Se ordenan los ususarios y se obtiene el top 10 con mas interaccioens (Menciones)

    # Liberamos memoria explícitamente para mejorar el rendimiento de la memoria
    del mention_counts

    return top_10_mentions

file_path = "farmers-protest-tweets-2021-2-4.json"
try:
    top_10_mentions = q3_time(file_path)
    print(top_10_mentions)
except FileNotFoundError:
    print(f"File not found: {file_path}. Please check the file path and try again.")
except json.JSONDecodeError as e:
    print(f"Error parsing JSON file: {e}")


[('narendramodi', 2261), ('Kisanektamorcha', 1836), ('RakeshTikaitBKU', 1639), ('PMOIndia', 1422), ('RahulGandhi', 1125), ('GretaThunberg', 1046), ('RaviSinghKA', 1015), ('rihanna', 972), ('UNHumanRights', 962), ('meenaharris', 925)]


Explicacion de los codigos para q1, q2 y q3 (**Tiempo de Ejecucion**)


**Explicacion de code q1**

Se importan las librerias

Importa el módulo json para trabajar con archivos JSON.

Importa defaultdict del módulo collections para crear diccionarios con valores por defecto, para la comprobacion de los datos y hacer mas eficiente el trabajo con grandes volumenes de datos.

from datetime import datetime, date: Importa datetime y date del módulo datetime para trabajar con fechas y horas.

from typing import List, Tuple: Importa List y Tuple del módulo typing para anotaciones de tipos, utilizados en la funcion de solucion.

Se define la funcion principal y se inicializa el conteo de tweets en 0

date_tweet_counts = defaultdict(lambda: defaultdict(int))

Se inicializa el archivo json en modo de lectura y se crea un ciclo para recorrer el documento
Luego se leen las lineas en busqueda de la fecha de los tweets y el usuario agregando al contador las menciones para cada usuario, controlando la busqueda con try/except con la finalidad de evitar el error en caso de que el formato de los datos no sea el correcto.

Se ordenan las fechas de los tweets considerando la cantidad de menciones de menera descendiente y se toman los primeros 10. Luego se entrega el resultado en una lista de tuplas, las cuales contienen la fecha y el usuario.

Finalmente, se llama al path del archivo y por medio de try/except se manejan los datos, considerando errores en el path del archivo y posibles errores en el formato de los datos.

**Explicacion de code q2**

Se utiliza un codigo similar al de la solucion en q1_time, pero se importa el modulo re para trabajar con expresiones regulares.

Luego de haber definido el contador de emojis en 0 se utiliza emoji_pattern = re.compile() para encontrar el unicode de los emojis.

Luego se realiza el mismo proceso utilizado en q1_time para recorrer el archivo json, encontrarl el top 10 y entregar los resultados.

Siempre considerando emplear try/except para manejo de errores

**Explicacion de code q3**

Se utiliza  una logica similar a las otras dos respuestas la diferencias son pocas y estan netamente enfocadas en la obtencion del username para encontrar al top 10 solicitado

mention_pattern = re.compile(r'@(\w+)'): Compila una expresión regular para encontrar menciones en el formato @username.

Se manejas las excepciones de la misma manera y se enfoca en obtener las menciones del usuario, contando directamente esto el el ciclo que recorre el archivo Json.

Como se menciono en la linea de codigo mention_pattern, se utiliza una expresion regular que se debe compilar una sola vez y luego solo se reutiliza recorriendo las tuplas dentro de las listas en el archivo con los datos.

Explicacion de los codigos para q1, q2 y q3 (**Uso de memoria**)


**Explicacion de code q1**

Se importan las librerias

Se importan las mismas librerias del q1_time, pero se agrega memory-profiler para medir el uso de memoria del script

Se utiliza el decorador @profile para medir el uso de memoria

Se define la funcion principal y se inicializa el conteo de tweets en 0

date_tweet_counts = defaultdict(lambda: defaultdict(int))

Se inicializa el archivo json en modo de lectura y se crea un ciclo para recorrer el documento Luego se leen las lineas en busqueda de la fecha de los tweets y el usuario agregando al contador las menciones para cada usuario, controlando la busqueda con try/except con la finalidad de evitar el error en caso de que el formato de los datos no sea el correcto.

Se ordenan las fechas de los tweets considerando la cantidad de menciones de menera descendiente y se toman los primeros 10. Luego se entrega el resultado en una lista de tuplas, las cuales contienen la fecha y el usuario.

Finalmente, se llama al path del archivo y por medio de try/except se manejan los datos, considerando errores en el path del archivo y posibles errores en el formato de los datos.

**Diferencias principales con q1_time**

Se evita la creacion de listas intemedias que pueden ser muy grandes dependiendo del archivo de datos

Minimiza el almacenamiento innecesario: Al usar date_tweet_counts.items() para la clasificación, asi evitar almacenar temporalmente una lista completa de fechas, lo que puede ser un ahorro significativo de memoria si el archivo JSON es grande.

**Explicacion de code q2**

Al igual que q2_time se importan las mismas librerais a diferencia del modulo emoji en su version 1.7 (Por tema de compatibilidad).

**Diferencias principales con q2_time**

La primera diferencia principal es el uso de la libreria emoji para una funcion extra que comprueba directamente si el caracter es un emoji por medio del unicode de la libreria. Esto optimiza el uso de memoria al ser una comparativa directa y entregar una respuesta booleana.

Adicionalmente, para mejorar el uso de memoria se utiliza filter en la funcion creada para verificar si el caracter es un emoji, lo que permite reducir el numero de operaciones realizadas.

Finalmente, a pesar que se cubre el mismo rango de caracteres que en q2_time, se validan, procesan y almacenan una menor cantidad de emojis.

**Explicacion de code q3**

Al igual que en q3_time se importan y trabajan los mismo modulos a diferencia de la libreria que se utiliza para medir el uso de memoria.

**Diferencias principales con q3_time**

La principal diferencia se encuentra en el uso de del mention_counts con el cual se libera directamente el uso de memoria, lo cual reduce el uso de memoria si un @usename cuenta con muchas menciones en diferentes tweets